In [1]:
import pickle
import numpy as np

In [2]:
def trial_utils(trial,start,end):
    losses = trial.losses()
    losses = [abs(i) for i in losses]
    losses = np.array(losses)
    best_indices = np.argwhere(losses == np.amin(losses))
    best_indices = best_indices.flatten().tolist()
    avg_score = losses[start:end].mean()
    

    best_score_id = trial.best_trial['tid']
    best_score = abs(trial.best_trial['result']['loss'])
    
    

    print('Best score:{} \n best score id:{} \n Average score[{},{}]:{}'.format(best_score,best_score_id,start,end,avg_score))
    print("all best scores idices {}".format(best_indices))
    print("-----------")
    all_best = {}
    for t in trial.trials:
        if abs(t['result']['loss']) == best_score:
            all_best[t['tid']]= t['misc']['vals']
#     return all_best    
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials,trials_from_docs
import random
from random import seed
def find_n_points(trial,number,strategy):
    """
    strategy :BEST - WORSE -RANDOM - BESTBEST
    based on  the strategy it returns N subset of provided trials
    """
    better_avg_index = []
    worse_avg_index =[]
    new_trial = []
    
    trial_loss = trial.losses()
    trial_loss = [abs(i) for i in trial_loss]
    
    avg_trial = np.array(trial_loss).mean()

    print("AVG score is {}".format(avg_trial))
    
    for x in enumerate(trial_loss):
        if x[1] > avg_trial:
            better_avg_index.append(x[0])
        else:
            worse_avg_index.append(x[0])
        
    if strategy == 'BEST':
        selected_list = better_avg_index
    if strategy == 'BESTBEST':
        selected_list = np.argsort(trial_loss)[0:number]
        
    if strategy == "WORSE":
        selected_list = worse_avg_index
    if strategy == 'MINMAX':
        score = int(number/2)
        list_1 = np.argsort(trial_loss)[0:score]
        list_2 = np.argsort(trial_loss)[-score:]
        print(list_1)
        print(list_2)
        selected_list = [*list_2 ,*list_1]
        print(selected_list)
    if strategy == "RANDOM":
        seed(1)
        selected_list = random.sample(range(1, len(trial.trials)), number)
#         print(selected_list)
        
    for i in range(len(trial.trials)):
        
        if i in selected_list[0:number]:
            new_trial.append(trial.trials[i])
    empty_trial = Trials()
    trial_merged = trials_from_docs(list(empty_trial)+new_trial)
    return trial_merged  

In [1]:
def find_n_initial(trial, N, good, bad):
    losses = trial.losses()
    losses_index = sorted(range(len(losses)), key=lambda k: losses[k], reverse=True)

    selected_list = []
    for i, v in enumerate(losses_index):
        if i >= N:
            break
        else:
            if i < N - good:
                selected_list.append(v)
            else:
                Bests = losses_index[-good:]
                selected_list = selected_list + Bests
                break

    new_trial = []
    for i in selected_list:
        new_trial.append(trial.trials[i])

    empty_trial = Trials()
    trial_merged = trials_from_docs(list(empty_trial) + new_trial)

    for i, v in enumerate(trial_merged.trials):

        need_to_change = trial_merged.trials[i]['tid']

        trial_merged.trials[i]['tid'] = i
        trial_merged.trials[i]['misc']['tid'] = i
        for key in v['misc']['idxs']:
            if v['misc']['idxs'][str(key)] == [need_to_change]:
                trial_merged.trials[i]['misc']['idxs'][str(key)] = [i]
    return trial_merged


# 0 initial 100 iteration

In [37]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials
import numpy as np




def objective(args):
    x= args
#     print("X is {} and loss is {}".format(x,x[0] ** 2))
    return {'loss': x[0] ** 2, 'status': STATUS_OK }


space  = [hp.uniform('x',-10,10)
]
trial = Trials()

best, new_trial= fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trial,
    rstate=np.random.RandomState(10))

print(best)
trial_utils(trial,0,100)

100%|██████████| 100/100 [00:00<00:00, 332.54it/s, best loss: 0.004264407779517356]
{'x': -0.06530243318221271}
Best score:0.004264407779517356 
 best score id:66 
 Average score[0,100]:22.23168055147256
all best scores idices [66]
-----------


In [ ]:
# trials = pickle.load(open("/home/dfki/Desktop/Thesis/hyperopt/trial_0initial_100iteration.p",'rb'))
# trial_utils(trials,0,100)

# 100 initial 100 iteration

In [ ]:
trials_100 = pickle.load(open("/home/dfki/Desktop/Thesis/hyperopt/trial_100initial_100iteration.p",'rb'))
trial_utils(trials_100,100,200)

In [8]:
best, trial_100_100initial= fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=200,
    trials=new_trial,
    rstate=np.random.RandomState(10))

print(best)
trial_utils(trial,100,200)

100%|██████████| 100/100 [00:00<00:00, 258.63it/s, best loss: 0.0007043068956300005]
{'x': 0.026538780974830034}
Best score:0.0007043068956300005 
 best score id:115 
 Average score[100,200]:12.7968807512244
all best scores idices [115]
-----------


# find the 1 good and 15 bad 

In [40]:
trial_16 = find_n_initial(new_trial,16,good=1,bad=15)
pickle.dump(trial_16, open('/home/dfki/Desktop/Thesis/hyperopt/trial_16_outof100.p', 'wb'))

In [41]:
trial_utils(trial_16,0,16)

Best score:0.004264407779517356 
 best score id:15 
 Average score[0,16]:69.86074901495861
all best scores idices [15]
-----------


In [42]:
best,trial_100_16initial= fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=116,
    trials=trial_16,
    rstate=np.random.RandomState(10))

print(best)
trial_utils(trial_100_16initial,16,116)

100%|██████████| 100/100 [00:00<00:00, 184.08it/s, best loss: 0.004264407779517356]
{'x': -0.06530243318221271}
Best score:0.004264407779517356 
 best score id:15 
 Average score[16,116]:11.307024973597787
all best scores idices [15]
-----------


# find the Best 50 out of 100

In [24]:
trial_50_BESTBEST_outof100 = find_n_points(new_trial,50,'BESTBEST')
# pickle.dump(trial_50_BESTBEST_outof100, open('/home/dfki/Desktop/Thesis/hyperopt/trial_50_BESTBEST_outof100.p', 'wb'))


AVG score is 22.23168055147256


In [44]:
trial_16.best_trial

{'state': 2,
 'tid': 15,
 'spec': None,
 'result': {'loss': 0.004264407779517356, 'status': 'ok'},
 'misc': {'tid': 15,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [15]},
  'vals': {'x': [-0.06530243318221271]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2019, 12, 9, 15, 16, 12, 951000),
 'refresh_time': datetime.datetime(2019, 12, 9, 15, 16, 12, 951000)}

In [26]:
best,trial_100_50initial_BEST= fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=150,
    trials=trial_50_BESTBEST_outof100,
    rstate=np.random.RandomState(10))

print(best)
trial_utils(trial_100_50initial_BEST,50,150)

100%|██████████| 100/100 [00:00<00:00, 301.58it/s, best loss: 0.004264407779517356]
{'x': -0.06530243318221271}
Best score:0.004264407779517356 
 best score id:66 
 Average score[50,150]:24.821388452254435
all best scores idices [29]
-----------


# Find 50 worset

In [34]:
trial_50_WORSE_outof100 = find_n_points(new_trial,50,'WORSE')

AVG score is 22.23168055147256


In [35]:
best,trial_100_50initial_worse= fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=150,
    trials=trial_50_WORSE_outof100,
    rstate=np.random.RandomState(10))

print(best)
trial_utils(trial_100_50initial_worse,50,150)

100%|██████████| 100/100 [00:00<00:00, 265.40it/s, best loss: 0.004264407779517356]
{'x': -0.06530243318221271}
Best score:0.004264407779517356 
 best score id:66 
 Average score[50,150]:38.60838984224944
all best scores idices [38]
-----------


In [45]:
trial.best_trial


{'state': 2,
 'tid': 66,
 'spec': None,
 'result': {'loss': 0.004264407779517356, 'status': 'ok'},
 'misc': {'tid': 66,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [66]},
  'vals': {'x': [-0.06530243318221271]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2019, 12, 9, 15, 16, 12, 951000),
 'refresh_time': datetime.datetime(2019, 12, 9, 15, 16, 12, 951000)}